multithreading:-  when multiple threads trying to change same variable we need to lock the process 

In [19]:

from threading import Thread
import time

db_val=0
def increase():
    
    global db_val
    loc_val=db_val
    loc_val+=1
    time.sleep(0.1)
    
    db_val=loc_val



if __name__ =="__main__":

    print("start value",db_val)
    thread1=Thread(target=increase)
    thread2=Thread(target=increase)


    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()


    print("end value",db_val)

start value 0
end value 1


in above code we run increase method twice so end value should be 2 but it is one because a race condition happened here. A race condition occurs when two or more threads can access shared data and they try to change it at the same time. 

to prevent race condition we use lock 

In [18]:

from threading import Thread,Lock
import time

db_val=0
def increase(lock):
    lock.acquire()
    global db_val
    loc_val=db_val
    loc_val+=1
    time.sleep(0.1)
 
    db_val=loc_val
    lock.release()


if __name__ =="__main__":
    lock=Lock()

    print("start value",db_val)
    thread1=Thread(target=increase,args=(lock,))
    thread2=Thread(target=increase,args=(lock,))


    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()


    print("end value",db_val)

start value 0
end value 2


Queues:- can be used for thread-safe/process-safe data exchanges and data processing both in a multithreaded and a multiprocessing environment.

In [25]:

from threading import Thread,Lock
import time
from queue import Queue


if __name__ =="__main__":
    q=Queue()
    q.put(1)
    q.put(2)
    q.put(3)
    q.put(4)
    print(q.get())
    print(q.get())
    print(q.get())
    print(q.get())
    

    print("end main")

1
2
3
4
end main


In [32]:
from threading import Thread, Lock, current_thread
from queue import Queue

def worker(q, lock):
    while True:
        value = q.get()  # blocks until the item is available

        # do stuff...
        with lock:
            # prevent printing at the same time with this lock
            print(f"in {current_thread().name} got {value}")
        # ...

        # For each get(), a subsequent call to task_done() tells the queue
        # that the processing on this item is complete.
        # If all tasks are done, q.join() can unblock
        q.task_done()


if __name__ == '__main__':
    q = Queue()
    num_threads = 10
    lock = Lock()

    for i in range(num_threads):
        t = Thread(name=f"Thread{i+1}", target=worker, args=(q, lock))
        t.daemon = True  # dies when the main thread dies
        t.start()
    
    # fill the queue with items
    for x in range(20):
        q.put(x)

    q.join()  # Blocks until all items in the queue have been gotten and processed.

    print('main done')


in Thread1 got 0
in Thread1 got 1
in Thread1 got 2
in Thread7 got 3
in Thread8 got 4
in Thread9 got 5
in Thread10 got 6
in Thread10 got 16
in Thread2 got 7
in Thread5 got 9
in Thread5 got 19
in Thread3 got 11
in Thread7 got 12
in Thread6 got 13
in Thread8 got 14
in Thread9 got 15
in Thread10 got 17
in Thread4 got 8
in Thread2 got 18
in Thread1 got 10
main done
